In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
import tensorflow as tf

In [2]:
data_train = pd.read_csv('../../../data/train_data.csv').values
data_test = pd.read_csv('../../../data/test_data.csv').values

# Build X and y
X_train = data_train[:, :-1]
y_train = data_train[:, -1]
X_test = data_test[:, :-1]
y_test = data_test[:, -1]

# Number of stocks in training data
n_att = X_train.shape[1]

# Neurons
n_neurons_1 = 64
n_neurons_2 = 32
n_neurons_3 = 16
n_neurons_4 = 8

# Placeholder
X = tf.placeholder(dtype=tf.float32, shape=[None, n_att])
Y = tf.placeholder(dtype=tf.float32, shape=[None])

# Initializers
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

# Hidden weights
W_hidden_1 = tf.Variable(weight_initializer([n_att, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
W_hidden_4 = tf.Variable(weight_initializer([n_neurons_3, n_neurons_4]))
bias_hidden_4 = tf.Variable(bias_initializer([n_neurons_4]))

# Output weights
W_out = tf.Variable(weight_initializer([n_neurons_4, 1]))
bias_out = tf.Variable(bias_initializer([1]))

# Hidden layer
hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
hidden_4 = tf.nn.relu(tf.add(tf.matmul(hidden_3, W_hidden_4), bias_hidden_4))

# Output layer (transpose!)
out = tf.transpose(tf.add(tf.matmul(hidden_4, W_out), bias_out))

# Cost function
mse = tf.reduce_mean(tf.squared_difference(out, Y))

# Optimizer
opt = tf.train.AdamOptimizer().minimize(mse)

# Saver
saver = tf.train.Saver()

In [3]:
with tf.Session() as sess:
    
    # Init
    sess.run(tf.global_variables_initializer())

    # Fit neural net
    batch_size = 256
    mse_train = []
    mse_test = []

    # Run
    epochs = 20
    for e in range(epochs):

        # Shuffle training data
        shuffle_indices = np.random.permutation(np.arange(len(y_train)))
        X_train = X_train[shuffle_indices]
        y_train = y_train[shuffle_indices]

        # Minibatch training
        for i in range(0, len(y_train) // batch_size):
            start = i * batch_size
            batch_x = X_train[start:start + batch_size]
            batch_y = y_train[start:start + batch_size]
            # Run optimizer with batch
            sess.run(opt, feed_dict={X: batch_x, Y: batch_y})

            # Show progress
            if np.mod(i, 50) == 0:
                # MSE train and test
                mse_train.append(sess.run(mse, feed_dict={X: X_train, Y: y_train}))
                mse_test.append(sess.run(mse, feed_dict={X: X_test, Y: y_test}))
                print('MSE Train: ', mse_train[-1])
                print('MSE Test: ', mse_test[-1])
                # Prediction
                pred = sess.run(out, feed_dict={X: X_test})
                print('pred: ', pred)
    save_path = saver.save(sess, "../models/model.ckpt")
    print("Model saved in path: %s" % save_path)

MSE Train:  1.0358046
MSE Test:  1.0465652
pred:  [[-0.13403483 -0.12651746 -0.10221933 ... -0.25424102 -0.3931911
  -0.27142626]]
MSE Train:  0.48916975
MSE Test:  0.4936012
pred:  [[ 0.96535903 -0.38144684  1.1601397  ... -0.5872416  -0.4921832
  -0.21764635]]
MSE Train:  0.3011873
MSE Test:  0.30368134
pred:  [[ 1.5607035  -0.45294952  2.1294014  ... -0.44786322 -0.33439568
  -0.2157014 ]]
MSE Train:  0.26297817
MSE Test:  0.26633155
pred:  [[ 1.3348784  -0.40262964  2.0557015  ... -0.4505312  -0.33085376
  -0.05896657]]
MSE Train:  0.24200055
MSE Test:  0.24375224
pred:  [[ 1.5770031  -0.4032377   2.4303026  ... -0.44677302 -0.33778572
  -0.06451935]]
MSE Train:  0.23686565
MSE Test:  0.23729578
pred:  [[ 1.68161    -0.3688413   2.8049514  ... -0.42190894 -0.33936286
   0.17451057]]
MSE Train:  0.22546846
MSE Test:  0.22497903
pred:  [[ 1.3466071  -0.34062836  2.3188236  ... -0.39972812 -0.31780872
   0.28485835]]
MSE Train:  0.21637529
MSE Test:  0.2160206
pred:  [[ 1.3018409  -0.

MSE Train:  0.19317594
MSE Test:  0.19391875
pred:  [[ 1.1798396  -0.46966252  1.8497581  ... -0.35129082 -0.41106722
  -0.03419837]]
MSE Train:  0.2011711
MSE Test:  0.2010895
pred:  [[ 0.93719244 -0.45743436  1.6255412  ... -0.33846596 -0.40905216
  -0.01989119]]
MSE Train:  0.20345303
MSE Test:  0.20314543
pred:  [[ 0.9207637  -0.43281516  1.6060503  ... -0.3269441  -0.40950617
  -0.02488317]]
MSE Train:  0.19312875
MSE Test:  0.19480975
pred:  [[ 1.1947794  -0.49965566  1.8988097  ... -0.37355626 -0.42587504
   0.11753666]]
MSE Train:  0.19184944
MSE Test:  0.1935587
pred:  [[ 1.1512371  -0.46562958  1.809081   ... -0.35240155 -0.4277495
  -0.01125421]]
MSE Train:  0.19632754
MSE Test:  0.19771034
pred:  [[ 1.0716587  -0.4478695   1.5628885  ... -0.34422266 -0.42919052
  -0.10671883]]
MSE Train:  0.19391868
MSE Test:  0.19380331
pred:  [[ 1.2297575  -0.4845074   1.8512594  ... -0.36406508 -0.43088004
   0.0569272 ]]
MSE Train:  0.19304062
MSE Test:  0.19394761
pred:  [[ 1.1611274e+

MSE Train:  0.19137481
MSE Test:  0.1927394
pred:  [[ 1.1498206  -0.441755    1.7976369  ... -0.3499687  -0.4516845
   0.00409943]]
MSE Train:  0.19041513
MSE Test:  0.1912443
pred:  [[ 1.0710279  -0.4285499   1.8325285  ... -0.36718333 -0.43562633
   0.00666411]]
MSE Train:  0.19406588
MSE Test:  0.1939501
pred:  [[ 1.3283157  -0.4672922   1.9599466  ... -0.37915567 -0.4378397
  -0.00938398]]
MSE Train:  0.19212958
MSE Test:  0.19147578
pred:  [[ 1.0069247  -0.45246103  1.824148   ... -0.37699294 -0.45135412
  -0.03338168]]
MSE Train:  0.19132055
MSE Test:  0.19048733
pred:  [[ 1.0737991  -0.44226062  1.9130285  ... -0.37572527 -0.45575887
   0.03609432]]
MSE Train:  0.19608581
MSE Test:  0.19612932
pred:  [[ 1.1446166  -0.46944532  2.0279808  ... -0.37763005 -0.44206977
   0.06736716]]
MSE Train:  0.19195245
MSE Test:  0.19229853
pred:  [[ 1.0579765  -0.45621628  1.7682564  ... -0.3534683  -0.45077062
  -0.13928345]]
MSE Train:  0.19052783
MSE Test:  0.19084202
pred:  [[ 1.102647   -

MSE Train:  0.19027068
MSE Test:  0.1906699
pred:  [[ 1.0900811  -0.47304     1.8069658  ... -0.38545313 -0.4484035
  -0.03247447]]
MSE Train:  0.18971485
MSE Test:  0.19055
pred:  [[ 1.2890987  -0.46994698  1.870944   ... -0.3974958  -0.45709404
   0.02209873]]
MSE Train:  0.19573534
MSE Test:  0.19743443
pred:  [[ 0.96122646 -0.44350612  1.5570283  ... -0.37344283 -0.44839668
  -0.02574547]]
MSE Train:  0.18958469
MSE Test:  0.19003643
pred:  [[ 1.2087935  -0.4502996   1.8311542  ... -0.37018397 -0.4575286
   0.03440204]]
MSE Train:  0.19370538
MSE Test:  0.19259325
pred:  [[ 1.2440184  -0.44669032  1.8701578  ... -0.38776934 -0.4414066
  -0.05563153]]
MSE Train:  0.18995383
MSE Test:  0.1894338
pred:  [[ 1.0733849  -0.46679986  1.7682192  ... -0.40294737 -0.4505266
   0.06007211]]
MSE Train:  0.18944237
MSE Test:  0.1889051
pred:  [[ 1.1210598  -0.47173542  1.8084313  ... -0.4058318  -0.45420453
   0.06854757]]
MSE Train:  0.19003353
MSE Test:  0.19024432
pred:  [[ 1.1804571  -0.468

MSE Train:  0.1908132
MSE Test:  0.19239275
pred:  [[ 1.1186581  -0.44820958  1.8418298  ... -0.37543103 -0.45450535
   0.01419023]]
MSE Train:  0.19014975
MSE Test:  0.18972544
pred:  [[ 1.1349723  -0.4798357   1.8700143  ... -0.3850317  -0.4768343
   0.04097058]]
MSE Train:  0.18978387
MSE Test:  0.19100823
pred:  [[ 1.2151865  -0.47228548  1.7627113  ... -0.3950415  -0.46422887
  -0.0279187 ]]
MSE Train:  0.19115362
MSE Test:  0.19103469
pred:  [[ 1.212093   -0.47731045  2.0276923  ... -0.38280812 -0.46209508
   0.06058463]]
MSE Train:  0.18875368
MSE Test:  0.18939666
pred:  [[ 1.1426411  -0.46850514  1.834343   ... -0.40629926 -0.47281027
   0.02292404]]
MSE Train:  0.18995696
MSE Test:  0.19069834
pred:  [[ 1.1292596  -0.4648802   1.7869649  ... -0.39277196 -0.47231412
   0.02500063]]
MSE Train:  0.18894586
MSE Test:  0.18918793
pred:  [[ 1.090436   -0.4602543   1.8131802  ... -0.39343065 -0.47393796
   0.00921327]]
MSE Train:  0.18923104
MSE Test:  0.18923303
pred:  [[ 1.0973104

MSE Train:  0.19350153
MSE Test:  0.19489188
pred:  [[ 1.0023838  -0.46397564  1.6609306  ... -0.37626773 -0.47012028
  -0.06039438]]
MSE Train:  0.18924658
MSE Test:  0.18961112
pred:  [[ 1.1741446  -0.45449764  1.8673737  ... -0.37561113 -0.4757645
   0.00932372]]
MSE Train:  0.18985996
MSE Test:  0.18982211
pred:  [[ 1.2559545  -0.48400968  1.9932934  ... -0.391613   -0.47541064
   0.02655101]]
MSE Train:  0.19105701
MSE Test:  0.19031206
pred:  [[ 1.1600026  -0.47124368  1.9914905  ... -0.39737916 -0.48461103
   0.06840201]]
MSE Train:  0.18928845
MSE Test:  0.1898407
pred:  [[ 1.0638951e+00 -4.6730602e-01  1.7375267e+00 ... -3.6634472e-01
  -4.6425420e-01 -2.0268559e-04]]
MSE Train:  0.19085152
MSE Test:  0.19077468
pred:  [[ 1.0943394  -0.46943092  1.7476056  ... -0.36433327 -0.46895522
   0.02926685]]
MSE Train:  0.19066669
MSE Test:  0.19133528
pred:  [[ 1.1003299  -0.47408074  1.7893947  ... -0.37809873 -0.4695121
  -0.05345009]]
MSE Train:  0.18881752
MSE Test:  0.18877846
pr